# SAP Automation using Python

In [5]:
import win32com.client
import subprocess
import time

In [ ]:
# # warning to display pop up message
# def warning():
#     from tkinter import messagebox
#     root = messagebox.showwarning("Warning", "This Automation bot is to view the material details which are already existing!")
#     # time for this message to stay
#     time.sleep(2)
#     # root.mainloop()
#     root.destroy()

In [7]:
# open SAP Logon
subprocess.Popen(r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection("USDEV",True)

# login
session = connection.Children(0)
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = "800" # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "JESAI"    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "jaisai2025"  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [ ]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nMIRO" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [ ]:
# # Entering Material Number
# session.findById("wnd[0]/usr/ctxtRMMG1-MATNR").text = "7100031" # Material Number
# session.findById("wnd[0]").sendVKey(0) # Enter
# time.sleep(1)


## Reading Excel file

In [ ]:
# pip install pandas openpyxl
import pandas as pd
from pathlib import Path

XLSX_PATH = Path(r"/mnt/data/Duty Invoice Automation.xlsx")  # change to your local path on Windows

KEEP_COLS = ["Date", "Entry Number", "PO Number", "Grand Total"]

# Read first sheet, coerce types
df = pd.read_excel(XLSX_PATH, sheet_name=0, dtype={"Entry Number": str, "PO Number": str})
# Standardize column names just in case of whitespace/casing
df.columns = [c.strip() for c in df.columns]

# Keep only the columns we care about (will raise if a column is missing)
missing = [c for c in KEEP_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

# Clean & format
out = (
    df[KEEP_COLS]
    .copy()
    .assign(
        # parse dates safely; coerce errors to NaT, then format to SAP-friendly string YYYYMMDD
        Date=lambda d: pd.to_datetime(d["Date"], errors="coerce").dt.strftime("%Y%m%d"),
        # strip spaces
        **{"Entry Number": df["Entry Number"].astype(str).str.strip(),
           "PO Number": df["PO Number"].astype(str).str.strip()}
    )
)

# Keep only rows that actually have something to post (you can tweak this rule)
out = out.dropna(how="all", subset=["Date", "Entry Number", "PO Number", "Grand Total"])

# Iterate row by row
records = out.to_dict(orient="records")
for r in records:
    print(r)
